First, let's import all the necessary dependencies.

In [ ]:
import xarray as xr
from cartopy import crs
import display
import matplotlib.pyplot as plt

Let's define some configuration variables.

In [ ]:
PROJECTION = crs.Miller()
AREA = {"latitude": slice(-5, 5), "longitude": slice(-120, -80)}
TIME_RANGE = ("1997-04", "1998-04")
DATASET_LOCATION = "s3://cmems-reanalysis/cmems_mod_glo_phy_my_0.083_P1D-m.zarr"

In the following lines we:
- establish a connection with the remote dataset
- show its metadata

It is worth noticing that, in the first operation, only the information necessary to gather the dataset's metadata (e.g. its sizes, dimensions, variables' type and attributes) is actually downloaded from its remote location to the local machine.

In [ ]:
ds = xr.open_dataset(DATASET_LOCATION, chunks={}, engine="zarr").astype("float32")
ds

Here we want to compute the average anomaly of the Sea Surface Height over a region of the Pacific Ocean during the 1997-1998 El Nino event, with respect to the whole time range available.

Note that the operations prescribed in the following lines are actually performed, and the resulting data downloaded, only when the `.compute()` method is met. For this reason, as a general rule to maximize efficiency, the `compute()` method should be placed after an operation which significantly reduce the amount of data, such as after selection and reduction operations.

In [ ]:
%%time

zos = ds.zos
zos_area = zos.sel(**AREA)
zos_mean = zos_area.mean(dim="time")
zos_mean

In [ ]:
zos_mean = zos_mean.compute()
zos_anomaly = zos_area - zos_mean
zos_anomaly_event = zos_anomaly.sel(time=slice(TIME_RANGE[0], TIME_RANGE[1]))
zos_anomaly_event_mean = zos_anomaly_event.mean(dim="time")

Let's plot the results on map.

In [ ]:
_, ax = plt.subplots(
    figsize=(10, 8),
    subplot_kw={"projection": PROJECTION},
)
zos_anomaly_event_mean.plot(
    ax=ax, 
    cmap="Blues", 
    transform=crs.PlateCarree(), 
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40, "label": "Sea Surface Height anomaly [m]"},
)
ax.coastlines()
ax.set_title("El Nino event (1997-1998)")
plt.show()

Finally, we want to compute the average monthly Surface Height anomaly for all the covered time-range, in order to highlight all occurred El Nino events.

The same considerations done before apply here.

In [ ]:
zos = ds.zos
zos_area = zos.sel(**AREA)
zos_monthly = zos_area.resample(time="1M").mean()
zos_monthly_spatial_mean = zos_monthly.mean(dim=["latitude", "longitude"])
zos_monthly_spatial_mean

In [ ]:
%%time

zos_monthly_spatial_mean = zos_monthly_spatial_mean.compute()
zos_mean = zos_monthly_spatial_mean.mean()
zos_anomaly = (zos_monthly_spatial_mean - zos_mean)

Let's plot the results as a time series.

In [ ]:
plt.plot(zos_anomaly['time'], zos_anomaly)
plt.xlabel('Year')
plt.ylabel('Sea Surface Height anomaly [m]')
plt.xticks(rotation=45)
plt.show()